In [1]:
# ==============================================================
# Example usage of trained Neural Networks
# Author: Elena González Prieto
#Date Modified: December 11th, 2025
# ==============================================================

import sys
import torch
import numpy as np
from sample_MoE import TaskSpecificMoE
import torch.nn.functional as F



In [2]:
# --------------------------------------------------------------
# Classification + Regression Model Only
# --------------------------------------------------------------

model = TaskSpecificMoE()

# Load the saved checkpoint for the classification model
checkpoint = torch.load(
    "./MoE_best_model.pt",  # replace with your file path
    map_location=torch.device("cpu")  # load on CPU for demonstration
)

# Load model weights
model.load_state_dict(checkpoint["model_state_dict"])

# Load normalization statistics (mean and std used during training)
input_mean = checkpoint["train_mean"]
input_std = checkpoint["train_std"]



/tmp/ipykernel_1127329/981484899.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(


In [3]:

# Data needs to be pre-processed in the exact way that the model was trained on. Input data should have the format 
# 0: log10(Age[Gyr] + 0.001) 1:log10(Rp[Rsun]+0.1) 2:log10(Vinf[km/s]+10) 3:ln(Mass1[MSUN]) 4:ln(Mass2[Msun])

# --------------------------------------------------------------
# Example input data
# --------------------------------------------------------------

# Example: say I have two 1 Msun stars, of age 5 Gyr, colliding at 100 km/s with periapsis of 0.5 Rsun: 
data = np.array([
    np.log10(5. + 0.001),
    np.log10(0.5 + 0.1),
    np.log10(100. + 10.),
    np.log(1.),
    np.log(1.)
], dtype=np.float32)

# Record the initial masses to transform regression outputs
initial_masses = np.exp(data[3]) + np.exp(data[4])

# --------------------------------------------------------------
# Normalization
# --------------------------------------------------------------

# Subtract the mean and divide by standard deviation computed on the training set.
# This ensures the new input data has the same scaling as the data the model was trained on.
normalized_data = (data - input_mean) / input_std


# Convert to torch tensor and add batch dimension
X = torch.tensor(normalized_data, dtype=torch.float32).unsqueeze(0)

In [6]:
# --------------------------------------------------------------
# Make Predictions
# --------------------------------------------------------------

# Classification labels include: 
# 0: mutual destruction, no stars left 
# 1: merger, one star left 
# 2: fly-by, two stars left
# 3: stripped star, 1 star left


# Regression quantities include (shape = [1, 3]): 
# M1,final [Msun] / Mtot,initial [Msun] (Note: you need to multiply by the initial mass of the system to recover the final mass in units of Msun)
# M2,final [Msun] / Mtot,initial [Msun] (Note: you need to multiply by the initial mass of the system to recover the final mass in units of Msun)
# Mejec,final [Msun] / Mtot,initial [Msun] (Here Mejec is the unbound mass)

classsification_pred, regression_pred = model(X)

predicted_class = torch.argmax(classsification_pred, dim=1).item()
predicted_values = regression_pred.squeeze(0).tolist()  # Converts tensor to a list


print(f"Predicted class: {predicted_class}")
print(f"Predicted Star 1 final mass in [MSUN]: {round(predicted_values[0] * initial_masses, 2)}")
print(f"Predicted Star 2 final mass in [MSUN]: {round(predicted_values[1] * initial_masses, 2)}")



Predicted class: 1
Predicted Star 1 final mass in [MSUN]: 1.9500000476837158
Predicted Star 2 final mass in [MSUN]: 0.0


In [17]:
Predicted class: 1
Predicted Star 1 final mass in [MSUN]: 2.7799999713897705
Predicted Star 2 final mass in [MSUN]: 0.0
    
Predicted class: 1
Predicted Star 1 final mass in [MSUN]: 2.440000057220459
Predicted Star 2 final mass in [MSUN]: 0.0 
    
    
Predicted class: 1
Predicted Star 1 final mass in [MSUN]: 14.680000305175781
Predicted Star 2 final mass in [MSUN]: 0.0
    

SyntaxError: invalid syntax (1645925153.py, line 1)